# 과제3 : 상품 카테고리별 수요 예측 및 재고 관리 인사이트
시간에 따른 상품 카테고리별 판매 트렌드를 분석하고, 계절성 패턴을 파악하여 향후 수요 예측과 재고 관리 전략을 수립하세요.

In [ ]:
# ============================================================
# 상품 카테고리별 수요 예측 & 재고 관리
# 1. 시간에 따른 카테고리별 판매 트렌드
# 2. 계절성 패턴 파악
# 3. 향후 수요 예측
# 4. 재고 관리 전략 수립
# ============================================================

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = lambda x: f"{x:,.2f}"

# -----------------------------
# 0) 데이터 로드 & 준비
# -----------------------------
orders   = pd.read_csv('./olist_orders_dataset.csv')
items    = pd.read_csv('./olist_order_items_dataset.csv')
products = pd.read_csv('./olist_products_dataset.csv')
cats_tr  = pd.read_csv('./product_category_name_translation.csv')

# 날짜형
orders['order_purchase_timestamp']      = pd.to_datetime(orders['order_purchase_timestamp'], errors='coerce')
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'], errors='coerce')

# 'delivered' 주문만
orders_ok = orders[(orders['order_status']=='delivered') & orders['order_purchase_timestamp'].notna()].copy()

# 카테고리명 영문화
products = products.merge(cats_tr, on='product_category_name', how='left')
products['category_en'] = products['product_category_name_english'].fillna(products['product_category_name']).fillna('unknown')

# 주문-아이템-상품 결합
oi = (items.merge(orders_ok[['order_id','order_purchase_timestamp']], on='order_id', how='inner')
           .merge(products[['product_id','category_en']], on='product_id', how='left'))
oi['ym']   = oi['order_purchase_timestamp'].dt.to_period('M').dt.to_timestamp()
oi['qty']  = 1  # Olist에서 아이템 행=수량 1로 취급
oi['rev']  = oi['price'] + oi['freight_value']

# -------------------------------------------------
# (1) 시간에 따른 카테고리별 판매 트렌드
# -------------------------------------------------
# 월별 집계
mth = (oi.groupby(['ym','category_en'])
         .agg(orders=('order_id','nunique'),
              units =('qty','sum'),
              revenue=('rev','sum'))
         .reset_index())

# 상위 카테고리(매출 기준) N 선택
TOP_N = 10
top_cats = (mth.groupby('category_en')['revenue'].sum()
              .sort_values(ascending=False).head(TOP_N).index.tolist())
mth_top = mth[mth['category_en'].isin(top_cats)].copy()

print("=== (1) 월별 판매 요약(상위 카테고리) ===")
display(mth_top.head())

# 추이 라인차트 (매출)
plt.figure(figsize=(11,5))
for c in top_cats:
    s = mth_top[mth_top['category_en']==c].set_index('ym')['revenue'].sort_index()
    plt.plot(s.index, s.values, label=c)
plt.title("(1) 월별 매출 추이 - 상위 카테고리")
plt.xlabel("월"); plt.ylabel("매출")
plt.legend(loc='upper left', ncol=2, frameon=False)
plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

# 비중 변화 (stacked area 느낌: 누적 막대)
share = (mth_top
         .pivot(index='ym', columns='category_en', values='revenue')
         .fillna(0).sort_index())
share_pct = share.div(share.sum(axis=1), axis=0)

plt.figure(figsize=(11,5))
bottom = np.zeros(len(share_pct))
idx = share_pct.index
for c in top_cats:
    vals = share_pct[c].reindex(idx).fillna(0).values
    plt.bar(idx, vals, bottom=bottom, width=25, align='center')
    bottom += vals
plt.title("(1) 카테고리별 매출 비중(월별)")
plt.ylabel("비중"); plt.ylim(0,1); plt.grid(axis='y', alpha=0.3)
plt.tight_layout(); plt.show()

In [ ]:
# -------------------------------------------------
# (2) 계절성 패턴 파악
# -------------------------------------------------
# 카테고리 × 월(1~12) 히트맵용 표
mth['month'] = mth['ym'].dt.month
heat = (mth[mth['category_en'].isin(top_cats)]
        .pivot_table(index='category_en', columns='month', values='revenue', aggfunc='mean')
        .fillna(0))
print("=== (2) 카테고리×월 평균 매출 ===")
display(heat.round(0))

# 간단 시각화: 카테고리별 월 평균 매출 라인 (상위 6개만)
plt.figure(figsize=(10,5))
for c in top_cats[:6]:
    s = mth[mth['category_en']==c].groupby('month')['revenue'].mean().reindex(range(1,13)).fillna(0)
    plt.plot(range(1,13), s.values, marker='o', label=c)
plt.title("(2) 월 평균 매출 패턴(계절성 힌트)"); plt.xlabel("월"); plt.ylabel("평균 매출")
plt.xticks(range(1,13)); plt.grid(alpha=0.3); plt.legend(frameon=False); plt.tight_layout(); plt.show()

# 계절성 강도 지표 (Hyndman식 근사): 1 - Var(rem)/Var(seasonal+rem)
seasonality_score = []
try:
    from statsmodels.tsa.seasonal import seasonal_decompose
    for c in top_cats:
        s = (mth[mth['category_en']==c]
                .set_index('ym')['revenue']
                .sort_index()
                .asfreq('MS'))   # 월시작 빈도
        if s.isna().mean() > 0.3 or len(s.dropna()) < 24:
            seasonality_score.append((c, np.nan))
            continue
        s = s.interpolate()
        res = seasonal_decompose(s, model='additive', period=12, two_sided=False, extrapolate_trend='freq')
        denom = np.var(res.seasonal + res.resid)
        num   = np.var(res.resid)
        sc = max(0, 1 - (num/denom)) if denom>0 else np.nan
        seasonality_score.append((c, sc))
    seas_df = pd.DataFrame(seasonality_score, columns=['category','seasonality_strength']).sort_values('seasonality_strength', ascending=False)
    print("=== (2) 계절성 강도(상위) ===")
    display(seas_df.head(10))
except Exception as e:
    print("(2) 계절성 분해 생략:", e)

In [ ]:
# -------------------------------------------------
# (3) 수요 예측 (상위 카테고리 중 3개)
# -------------------------------------------------
sel = top_cats[:3]  # 예측 대상
h  = 6              # 예측월

def sarima_forecast(series, h=6):
    """간단 SARIMA(1,1,1)(1,1,1,12) 예측. 데이터 부족/실패 시 None 반환"""
    try:
        from statsmodels.tsa.statespace.sarimax import SARIMAX
        train = series.asfreq('MS').interpolate()
        model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
        fit   = model.fit(disp=False)
        fcst  = fit.get_forecast(steps=h)
        pred  = fcst.predicted_mean
        conf  = fcst.conf_int()
        return pred, conf
    except Exception as e:
        print("SARIMA error:", e)
        return None, None

for c in sel:
    s = (mth[mth['category_en']==c]
         .set_index('ym')['units']   # 단위: 판매수량으로 예측
         .sort_index())
    if len(s) < 18:
        print(f"(3) {c}: 데이터가 부족하여 예측 생략")
        continue
    pred, conf = sarima_forecast(s, h=h)
    if pred is None:
        continue

    # 그래프 (히스토리 + 예측)
    plt.figure(figsize=(10,4))
    plt.plot(s.index, s.values, label='history')
    plt.plot(pred.index, pred.values, label='forecast')
    if conf is not None:
        plt.fill_between(pred.index, conf.iloc[:,0].values, conf.iloc[:,1].values, alpha=0.2)
    plt.title(f"(3) {c} - 판매수량 예측(+{h}개월)")
    plt.xlabel("월"); plt.ylabel("수량")
    plt.grid(alpha=0.3); plt.legend(frameon=False); plt.tight_layout(); plt.show()

In [ ]:
# -------------------------------------------------
# (4) 재고 관리 전략 (간단 계산: 안전재고 & 재주문점)
#   - 일별 수요 기반 ROP = dbar*L + z*σ_d*sqrt(L),  z=1.645(95% 서비스레벨)
#   - 카테고리별 리드타임은 주문→배송 완료 평균(일)로 근사
# -------------------------------------------------
# 일별 수요(수량)
oi['date'] = oi['order_purchase_timestamp'].dt.date
daily = (oi.groupby(['date','category_en'])['qty'].sum().reset_index())
# 카테고리별 리드타임(일) 근사: 주문-배송 완료 (아이템→주문 결합)
tmp = (oi[['order_id','category_en']].drop_duplicates()
          .merge(orders_ok[['order_id','order_purchase_timestamp','order_delivered_customer_date']], on='order_id', how='left'))
tmp['lead_days'] = (tmp['order_delivered_customer_date'] - tmp['order_purchase_timestamp']).dt.total_seconds()/86400

L = tmp.groupby('category_en')['lead_days'].mean().rename('L').to_frame()
dstat = (daily.groupby('category_en')['qty']
              .agg(dbar='mean', dstd='std')
              .reset_index()).merge(L, left_on='category_en', right_index=True, how='left')

dstat = dstat[dstat['category_en'].isin(sel)]  # 예측 대상과 동일 그룹만 샘플 계산
z = 1.645  # 95%
dstat['safety_stock'] = (z * dstat['dstd'] * np.sqrt(dstat['L'].clip(lower=1)))
dstat['ROP'] = (dstat['dbar'] * dstat['L'].clip(lower=1)) + dstat['safety_stock']

print("=== (4) 재고 전략 (예: 상위 3개 카테고리) ===")
print("가정: 일별 수요/평균 리드타임 기반, 서비스레벨=95%")
display(dstat[['category_en','dbar','dstd','L','safety_stock','ROP']].round(2))

print("\n[전략 가이드]")
print("- 성수기(계절성 강한 월 이전)에는 안전재고를 일시적으로 상향(예: z=2.05, 97.5%)")
print("- 리드타임 긴 카테고리는 전진배치/사전발주로 L 단축 → ROP 낮추기")
print("- 비수기에는 z를 낮추고 재고회전이 느린 SKU는 단계적 감량")
print("- 지역 물류(과제2)와 결합: 지연 높은 권역엔 버퍼 확대, 빠른 권역엔 회전 우선")

# -------------------------------------------------
# 요약 텍스트
# -------------------------------------------------
print("\n=== 요약 ===")
print(f"- 상위 {TOP_N} 카테고리 중심으로 월별 매출/비중 추이를 확인했습니다.")
print("- 월 평균 패턴과 계절성 지표(가능 시)를 통해 성수기 후보 월을 특정했습니다.")
print(f"- 예측 대상 {len(sel)}개 카테고리에 대해 SARIMA(+{h}개월) 수요 예측을 시각화했습니다.")
print("- 안전재고·재주문점(ROP)을 간단 산식으로 산출해 운영 전략 방향을 제시했습니다.")
